Analysis reads in title and abstract data from file, runs mordecai and returns a data frame with unique locations for each entry

*Note* is it posisble to save as an sqlite file instead?

In [1]:
# Set up mordecai geoparser
from mordecai import Geoparser
geo = Geoparser()

Models path: C:\Users\deviv\anaconda3\envs\mordecai_env_3.8\lib\site-packages\mordecai\models\


In [2]:
# Import other packages
import pandas as pd
import pickle
import os

In [ ]:
## Read in geoparsing data
df = pd.read_csv("\\".join([os.getcwd(),"data","raw-data","test-file.csv"]))

# combine title and abstract text into one variable
df['text'] = df['title'] + df['abstract']

# summarise
df.head()
df.shape

In [ ]:
## Geoparse the text and add results as a new column "geoparse"

df["geoparse"] = [geo.geoparse(x) for x in df["text"].astype('str')]

In [ ]:
## Format dataframe

# create new formatted dataframe: df_clean
def flat_df(df):
    df_geo = df[df["geoparse"].str.len() != 0] #subset where geoparse string is not empty
    df_geo = df_geo.explode('geoparse') #Transforms each element of a list to a row and replicates index and all other columns. When more than one place name appears it creates more than one row.
    df_geo = pd.concat([df_geo.drop(['geoparse'], axis=1), df_geo['geoparse'].apply(pd.Series)], axis=1) #Extract from dic
    df_geo = pd.concat([df_geo.drop(['geo'], axis=1), df_geo['geo'].apply(pd.Series)], axis=1)
    df_geo = df_geo[df_geo['lat'].notnull()] #Removing empty latitude rows
    df_geo.lat = df_geo.lat.astype(float) #Transforms to float
    df_geo.lon =df_geo.lon.astype(float) #Transforms to float
    return df_geo
    
df_clean = flat_df(df)
print(df_clean.dtypes)
print(df_clean[0:5])
df_clean.head(5)

In [ ]:
## Save as a .csv
df_clean.to_csv("\\".join([os.getcwd(),"outputs","geoparsed-records.csv"]), index=False)